In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [2]:
// Create network
val model = Network("model")

// Block 1: nodes with mixed scheduling
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)   // Processor Sharing
val node3 = Queue(model, "Queue2", SchedStrategy.DPS)  // Discriminatory Processor Sharing

// Block 2: classes with different populations
val jobclass1 = ClosedClass(model, "Class1", 2, node1, 0)  // 2 jobs
val jobclass2 = ClosedClass(model, "Class2", 1, node1, 0)  // 1 job

println("Network created with mixed scheduling:")
println("  ${node1.name}: Delay node")
println("  ${node2.name}: PS (Processor Sharing)")
println("  ${node3.name}: DPS (Discriminatory Processor Sharing)")
println("\nClass populations: Class1=${jobclass1.population}, Class2=${jobclass2.population}")

Network created with mixed scheduling:
  Delay: Delay node
  Queue1: PS (Processor Sharing)
  Queue2: DPS (Discriminatory Processor Sharing)

Class populations: Class1=2.0, Class2=1.0


In [ ]:
// Set service distributions
node1.setService(jobclass1, Exp(3.0))    // Delay, Class1: rate=3
node1.setService(jobclass2, Exp(0.5))    // Delay, Class2: rate=0.5

// PS Queue - weights are ignored in PS scheduling
val w1_ps = 5.0  // This weight is ignored since the node is PS
val w2_ps = 1.0  // This weight is ignored since the node is PS
node2.setService(jobclass1, Exp(0.1), w1_ps)
node2.setService(jobclass2, Exp(1.0), w2_ps)

// DPS Queue - weights matter for service differentiation
val w1_dps = 1.0  // Lower weight for Class1
val w2_dps = 5.0  // Higher weight for Class2 (gets priority)
node3.setService(jobclass1, Exp(0.1), w1_dps)
node3.setService(jobclass2, Exp(1.0), w2_dps)

println("Service parameters configured:")
println("  Delay: Class1=Exp(3), Class2=Exp(0.5)")
println("  Queue1 (PS): Class1=Exp(0.1), Class2=Exp(1) - weights ignored")
println("  Queue2 (DPS): Class1=Exp(0.1,w=$w1_dps), Class2=Exp(1,w=$w2_dps) - Class2 has priority")

In [4]:
// Set up routing matrix
val P = model.initRoutingMatrix()

// Class1 routing: probabilistic from Delay (30% Queue1, 70% Queue2)
P.set(jobclass1, jobclass1, node1, node2, 0.3)  // Delay -> Queue1 (30%)
P.set(jobclass1, jobclass1, node1, node3, 0.7)  // Delay -> Queue2 (70%)
P.set(jobclass1, jobclass1, node2, node1, 1.0)  // Queue1 -> Delay (100%)
P.set(jobclass1, jobclass1, node3, node1, 1.0)  // Queue2 -> Delay (100%)

// Class2 routing: probabilistic from Delay (70% Queue1, 30% Queue2)
P.set(jobclass2, jobclass2, node1, node2, 0.7)  // Delay -> Queue1 (70%)
P.set(jobclass2, jobclass2, node1, node3, 0.3)  // Delay -> Queue2 (30%)
P.set(jobclass2, jobclass2, node2, node1, 1.0)  // Queue1 -> Delay (100%)
P.set(jobclass2, jobclass2, node3, node1, 1.0)  // Queue2 -> Delay (100%)

model.link(P)

println("Routing configured:")
println("  Class1: 30% Queue1, 70% Queue2 (more traffic to DPS queue)")
println("  Class2: 70% Queue1, 30% Queue2 (more traffic to PS queue)")
println("  All queues return 100% to Delay")

Routing configured:
  Class1: 30% Queue1, 70% Queue2 (more traffic to DPS queue)
  Class2: 70% Queue1, 30% Queue2 (more traffic to PS queue)
  All queues return 100% to Delay


In [5]:
// Configure multiple solvers for comparison
println("=== Multi-Solver Analysis ===")

val solvers = arrayOf(
    CTMC(model, "verbose", 1),
    JMT(model, "verbose", 1, "samples", 10000, "seed", 23000),
    FLD(model, "verbose", 1),
    MVA(model, "verbose", 1)
)

println("Configured ${solvers.size} solvers for comparison")

=== Multi-Solver Analysis ===
Configured 4 solvers for comparison


In [ ]:
import jline.solvers.NetworkAvgTable

// Solve with each solver and compare results
val avgTables = mutableListOf<NetworkAvgTable>()

for (solver in solvers) {
    println("\n=== SOLVER: ${solver.getName().replace("Solver", "")} ===")
    try {
        val avgTable = solver.getAvgTable()
        avgTables.add(avgTable)
        avgTable.print()
    } catch (e: Exception) {
        println("Solver failed: ${e.message}")
    }
}